In [ ]:
# This notebook provides an example of importing a trained Caffe model and evaluating the performance of the model
# on a subset of the ImageNet data set.  The code below makes several assumptions:
#
# 1.) The ImageNet example files obtained from the SAS support web site have been unpacked in the 
#     directory $IMAGENET_EXAMPLE
#
# 2.) You have already started a CAS server and you know the port number associated with that server
#
# 3.) Your operating system is either Linux or Windows
#
# 4.) You have obtained the model definition file models.zip from the SAS support web site and
#     and unpacked the contents in the directory $IMAGENET_EXAMPLE 
#
# 5.) You have obtained model parameter file(s) from the SAS support web site (e.g. vgg16.zip) and
#     unpacked the contents in $MODEL_ROOT
#
# 6.) You have a downloaded a subset of the ImageNet data set to $IMAGE_ROOT/$IMAGE_CLASS where $IMAGE_CLASS is
#     one of the image classes defined by ImageNet.  This program uses $IMAGE_CLASS=ladybug.
#
# 7.) Windows paths are specified in this form
#          <drive letter>:\\directory1\\subdirectory1\\... (e.g. c:\\imagenet\\images)
#
# 8.) Linux paths are specified in this form
#           /directory1/subdirectory1/... (e.g. /imagenet/images)

In [ ]:
# import packages/functions needed
from swat import *
import sys

In [ ]:
# define variables used to control example
#    imagePath = path to directory where ImageNet images are stored
#    modelPath = path to directory where converted Caffe model(s) are stored
#    modelName = name of Caffe model file
#    casHost = full name of CAS server host (e.g. cashost.xxx.yyy.zzz)
#    casPort = port associated with CAS server (e.g. 12345)
#    serverOsType = CAS server operating system type (WINDOWS or LINUX) 

imagePath = "<your_image_path>"             # string
modelPath = "<your_model_path>"             # string
modelName = "<LeNet_model_file_name>"       # string, valid choices: ResNet50, ResNet101, ResNet152, VGG16, VGG19
casHost = "<CAS_host_name>"                 # string
casPort = <CAS_port>                        # integer
serverOsType = "<CAS server OS>"            # string - either WINDOWS or LINUX

if (serverOsType == 'WINDOWS'):
    pathSeparator = '\\'
elif (serverOsType == 'LINUX'):
    pathSeparator = '/'
else:
    sys.exit("ERROR: unsupported operating system")

In [ ]:
# import pre-defined models
from model_defs import *

In [ ]:
# create CAS session and load action sets
s = CAS(casHost, casPort )
s.sessionprop.setsessopt(caslib='CASUSER', timeout=31535000)
s.loadactionset('image')
s.loadactionset('deepLearn')

In [ ]:
# load subset of ImageNet data
s.loadImages(casout={"name":"imagenet", "replace":True}, path=imagePath)

In [ ]:
# resize images to 224 x 224 pixels
s.processImages(table=dict(name="imagenet"), 
                casout={"name":"imagenet_resize", "replace":True}, 
                imagefunctions=[dict(functionoptions=dict(functiontype='RESIZE',w=224, h=224))])

In [ ]:
# add library for Caffe models
s.table.addcaslib(activeonadd=False,datasource={'srctype':'path'},
                  name='modelhome',path=modelPath,subdirectories=True)

In [ ]:
# load labels for output neurons
s.table.loadtable(caslib='modelhome',
                  casout={'caslib':'CASUSER','name':'imagenet_labels',"replace":True},
                  importoptions={'filetype':'basesas'},
                  path='newlabel.sas7bdat',
                  promote=False)

In [ ]:
# instantiate model
if (modelName == 'ResNet50'):
    ResNet50_Model(s,inputCropType='NONE')
    modelFile = modelPath + pathSeparator + "ResNet-50-model.caffemodel.h5"
elif (modelName == 'ResNet101'):
    ResNet101_Model(s,inputCropType='NONE')
    modelFile = modelPath + pathSeparator + "ResNet-101-model.caffemodel.h5"
elif (modelName == 'ResNet152'):
    ResNet152_Model(s,inputCropType='NONE')
    modelFile = modelPath + pathSeparator + "ResNet-152-model.caffemodel.h5"
elif (modelName == 'VGG16'):
    VGG16_Model(s,inputCropType='NONE')
    modelFile = modelPath + pathSeparator + "VGG_ILSVRC_16_layers.caffemodel.h5"
elif (modelName == 'VGG19'):
    VGG19_Model(s,inputCropType='NONE')
    modelFile = modelPath + pathSeparator + "VGG_ILSVRC_19_layers.caffemodel.h5"
else:
    print('ERROR: You specified a model that is not yet supported')

In [ ]:
# load weights from Caffe model
s.dlimportmodelweights(model=modelName,
                       modelWeights={'caslib':'CASUSER','name':'tkdlWeights','replace':1},
                       labelTable={'caslib':'CASUSER','name':'imagenet_labels','vars':['levid','levname']},
                       formatType="CAFFE",
                       weightFilePath=modelFile)

In [ ]:
# generate the SAS code for applying labels to images
pString1 = "_label_ = "
pLabel = "'ladybug"
pString2 = "';"
programString = pString1 + pLabel.ljust(51) + pString2

In [ ]:
# score the validation data
s.dlscore(model=modelName,
          initWeights={'caslib':'CASUSER','name':'tkdlWeights'},
          table={'caslib':'CASUSER','name':'imagenet_resize',
                 'compvars':'_label_','comppgm':programString},
          copyvars=['_label_'],
          topProbs = 5,
          miniBatchSize = 3,
          casout={'caslib':'CASUSER','name':'ScoredValidation', 'replace':True})

In [ ]:
# terminate CAS session
s.close()